In [74]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import pymongo
from datetime import datetime
import json 
import talib as ta

myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["stoccs"]
print("ok")

ok


In [79]:
mycol = mydb["data"]
now = datetime.now()
tickerlist = ["AAPL", "MSFT", "FB", "0700.hk"]

def newdata():
    mycol.drop({})
    for tickername in tickerlist:
        ticker = yf.download(tickername, period = "1y")
        ticker['SMA10'] = ta.SMA(ticker['Close'], timeperiod=10)
        ticker['SMA20'] = ta.SMA(ticker['Close'], timeperiod=20)
        ticker['SMA50'] = ta.SMA(ticker['Close'], timeperiod=50)
        if not ticker.empty:
            df = pd.DataFrame(ticker)
            data = df.reset_index()
            json_data = json.loads(data.to_json(date_format="iso"))
            data_list = []
            for j in range(len((json_data["Date"]))):
                row = {}
                for col in data.columns:
                    row[col] = json_data[col][str(j)]
                    if col == "Date":
                        row[col] = datetime.strptime(json_data[col][str(j)], '%Y-%m-%dT%H:%M:%S.%fZ')
                data_list.append(row)
            
            ticker_data = []
            for row in data_list:
                ticker_data.append(row)
                
            mycol.insert_one({tickername: ticker_data})

# def update():
#     for ticker in tickerlist:
#         result = mycol.find({}, {"ticker.Date": 1})
#         print(result)

newdata()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [67]:
mycol = mydb["data"]

findstuff = mycol.aggregate([
    { "$project": {
        "MSFT": {
            "$filter": {
                "input": "$MSFT",
                "as": "item",
                "cond": 
                    {"$and": [{ "$gt": ["$$item.Date", datetime(2022, 1, 27, 0, 0)]}]}
                }
            }
        }
    }
])

result = [x for x in findstuff if x["MSFT"] is not None]

print(result)

[{'_id': ObjectId('61fe20befc5c7b5f3638e6ec'), 'MSFT': [{'Date': datetime.datetime(2022, 1, 28, 0, 0), 'Open': 300.2300109863, 'High': 308.4800109863, 'Low': 294.450012207, 'Close': 308.2600097656, 'Adj Close': 308.2600097656, 'Volume': 49743700}, {'Date': datetime.datetime(2022, 1, 31, 0, 0), 'Open': 308.950012207, 'High': 312.3800048828, 'Low': 306.3699951172, 'Close': 310.9800109863, 'Adj Close': 310.9800109863, 'Volume': 46444500}, {'Date': datetime.datetime(2022, 2, 1, 0, 0), 'Open': 310.4100036621, 'High': 310.6300048828, 'Low': 305.1300048828, 'Close': 308.7600097656, 'Adj Close': 308.7600097656, 'Volume': 40894300}, {'Date': datetime.datetime(2022, 2, 2, 0, 0), 'Open': 309.6300048828, 'High': 315.1199951172, 'Low': 308.8800048828, 'Close': 313.4599914551, 'Adj Close': 313.4599914551, 'Volume': 36636000}, {'Date': datetime.datetime(2022, 2, 3, 0, 0), 'Open': 309.4899902344, 'High': 311.2300109863, 'Low': 299.9599914551, 'Close': 301.25, 'Adj Close': 301.25, 'Volume': 43730000}, 